In [1]:
import pandas as pd
import numpy as np
import subprocess
import os


In [4]:
phenotypes = pd.read_csv('../data/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.gz', compression='gzip', sep='\t')
filtered_phenotypes = phenotypes[~phenotypes['Chr'].isin(['X', 'Y', 'M'])]
to_calc = filtered_phenotypes[['TargetID', 'Chr']]
#to_calc = to_calc[to_calc['Chr'].isin(['19', '20', '21'])]
to_calc

,TargetID,Chr
23,ENSG00000225267.1,21
35,ENSG00000228184.1,21
52,ENSG00000126457.15,19
55,ENSG00000099804.2,19
66,ENSG00000130734.4,19
...,...,...
23643,ENSG00000142396.4,19
23646,ENSG00000125966.7,20
23672,ENSG00000254995.4,20
23683,ENSG00000232241.1,20


In [ ]:
# Define constants for paths and parameters
RSCRIPT_PATH = "Rscript"
FUSION_SCRIPT = "fusion_twas-master/FUSION.compute_weights.R"
BFILE_TEMPLATE = "../data/LDREF_filtered/1000G.EUR.{chr}_filtered"  # Placeholder for chromosome
TMP_DIR = "temp_files"
OUTPUT_DIR = "compute_weights_out"
MODELS = "enet"
PLINK_PATH = "plink.exe"
GCTA_PATH = "fusion_twas-master/gcta_nr_robust.exe"
PHENO_DIR = "../data/gene_expressions"  # Directory with phenotype files
VERBOSE = 2

def compute_weights(gene, chromosome):
    """Runs the FUSION TWAS pipeline for a specific gene and chromosome."""
    # Construct file paths
    bfile = BFILE_TEMPLATE.format(chr=chromosome)
    tmp_file_prefix = os.path.join(TMP_DIR, f"test_chr{chromosome}_{gene}")
    output_path = os.path.join(OUTPUT_DIR, f"{gene}_chr{chromosome}")
    pheno_file = os.path.join(PHENO_DIR, f"{gene}.txt")
    
    # Construct the command
    command = [
        RSCRIPT_PATH, FUSION_SCRIPT,
        "--bfile", bfile,
        "--tmp", tmp_file_prefix,
        "--out", output_path,
        "--models", MODELS,
        "--PATH_plink", PLINK_PATH,
        "--PATH_gcta", GCTA_PATH,
        "--verbose", str(VERBOSE),
        "--pheno", pheno_file
    ]
    clear_command = ['rm', '-rf', 'temp_files/*']

    # Print command for debugging
    print(f"Running: {' '.join(command)}")
    
    # Run the command
    try:
        subprocess.run(command, check=True)
        subprocess.run(clear_command, check=True)
        
    except subprocess.CalledProcessError as e:
        print(f"Error running command for gene {gene}, chromosome {chromosome}: {e}")



for i in to_calc.index:
    gene_data = to_calc.loc[i]
    compute_weights(gene_data['TargetID'], gene_data['Chr'])

Running: Rscript fusion_twas-master/FUSION.compute_weights.R --bfile ../data/LDREF_filtered/1000G.EUR.21_filtered --tmp temp_files\test_chr21_ENSG00000225267.1 --out compute_weights_out\ENSG00000225267.1_chr21 --models enet --PATH_plink plink.exe --PATH_gcta fusion_twas-master/gcta_nr_robust.exe --verbose 2 --pheno ../data/gene_expressions\ENSG00000225267.1.txt
Running: Rscript fusion_twas-master/FUSION.compute_weights.R --bfile ../data/LDREF_filtered/1000G.EUR.21_filtered --tmp temp_files\test_chr21_ENSG00000228184.1 --out compute_weights_out\ENSG00000228184.1_chr21 --models enet --PATH_plink plink.exe --PATH_gcta fusion_twas-master/gcta_nr_robust.exe --verbose 2 --pheno ../data/gene_expressions\ENSG00000228184.1.txt
Running: Rscript fusion_twas-master/FUSION.compute_weights.R --bfile ../data/LDREF_filtered/1000G.EUR.19_filtered --tmp temp_files\test_chr19_ENSG00000126457.15 --out compute_weights_out\ENSG00000126457.15_chr19 --models enet --PATH_plink plink.exe --PATH_gcta fusion_twas